# 📘 RAG with Qdrant, Google Gemini, and LangChain

This project demonstrates how to build a **Retrieval-Augmented Generation (RAG)** pipeline using **LangChain Expression Language (LCEL)**, **Qdrant** as a vector store, and **Google Gemini** as the LLM. It allows you to query knowledge from your own PDF documents, combining **retrieval** and **generation** in a clean, modular pipeline.

In [1]:
import os
from dotenv import load_dotenv

# Load env
load_dotenv()
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [2]:
from utils.loader import load_and_split_pdfs
from utils.embeddings import get_embedding_model
from utils.ingest import ingest_documents
from utils.retriever import get_retriever, format_docs
from utils.llm import get_llm_model
from utils.prompt import get_rag_prompt, get_chat_rag_prompt
from langchain.schema.runnable import RunnableLambda
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Documents
PDFS = [
    "data/eldoria_history.pdf",
    "data/quantum_drive_specs.pdf",
    "data/zirconia_recipes.pdf"
]

# Step 1: Load + split PDFs
docs = load_and_split_pdfs(PDFS)

# Step 2: Embeddings
embedding_model = get_embedding_model(GOOGLE_API_KEY)

# Step 3: Ingest into Qdrant
ingest_documents(QDRANT_URL, QDRANT_API_KEY, QDRANT_COLLECTION, embedding_model, docs)

# Step 4: Build retriever + LLM
retriever = get_retriever(QDRANT_URL, QDRANT_API_KEY, QDRANT_COLLECTION, embedding_model, k=3)
llm = get_llm_model(GOOGLE_API_KEY, model="models/gemini-flash-latest")

# Step 5: Prompt
prompt = get_chat_rag_prompt()

# Step 6: Build RAG pipeline with LCEL
rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Example queries
queries = [
    "What led to the fall of the Kingdom of Eldoria?",
    "How does the Starlink Quantum Drive create thrust?",
    "What are the ingredients and effects of Crystal Soup from Zirconia?"
]

for q in queries:
    print("\n[QUERY]", q)
    result = rag_chain.invoke(q)
    print("[ANSWER]", result)

📚 Loaded and split 3 PDF(s) into 9 chunks
ℹ️ Using existing Qdrant collection 'my_rag_collection'


E0000 00:00:1759278229.102128   77990 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ Collection 'my_rag_collection' already has 72 vectors. Skipping ingestion.
ℹ️ Using existing Qdrant collection 'my_rag_collection'

[QUERY] What led to the fall of the Kingdom of Eldoria?


E0000 00:00:1759278230.630560   78062 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


[ANSWER] The fall of Eldoria was caused by internal strife and civil war among rival noble houses, the rise of the Great Shadow War (an invasion by creatures of darkness), and the withdrawal of the dragons, who were once allies.

[QUERY] How does the Starlink Quantum Drive create thrust?
[ANSWER] The Starlink Quantum Drive creates a localized warp bubble by manipulating entangled states within the Quantum Resonance Chamber (QRC). Within this bubble, space contracts in front of the ship and expands behind it, propelling the craft forward without traditional thrust.

[QUERY] What are the ingredients and effects of Crystal Soup from Zirconia?
[ANSWER] The ingredients of Crystal Soup are ground luminous crystals and plasma herbs. Locals believe the soup enhances stamina and sharpens the senses.


📚 Loaded and split 3 PDF(s) into 9 chunks
ℹ️ Using existing Qdrant collection 'my_rag_collection'
E0000 00:00:1759277752.255911   75207 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
✅ Ingested 9 documents into collection 'my_rag_collection'
ℹ️ Using existing Qdrant collection 'my_rag_collection'

[QUERY] What led to the fall of the Kingdom of Eldoria?
E0000 00:00:1759277758.631478   75765 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
[ANSWER] The fall of Eldoria was caused by internal strife and civil war among rival noble houses, which began in the 14th century. This was further destabilized by the rise of the Great Shadow War, where creatures of darkness invaded the borderlands, and the withdrawal of dragons, who were once allies.

[QUERY] How does the Starlink Quantum Drive create thrust?
[ANSWER] The Starlink Quantum Drive creates a localized warp bubble by manipulating entangled states within the Quantum Resonance Chamber (QRC). Within this bubble, space contracts in front of the ship and expands behind it, propelling the craft forward without traditional thrust.

[QUERY] What are the ingredients and effects of Crystal Soup from Zirconia?
[ANSWER] The ingredients of Crystal Soup are ground luminous crystals and plasma herbs. Locals believe the soup enhances stamina and sharpens the senses.

